In [ ]:
#https://medium.com/@vasista/sentiment-analysis-using-svm-338d418e3ff1

from google.colab import files
uploaded = files.upload()

Saving test_clean.xlsx to test_clean (1).xlsx
Saving trainG_clean.xlsx to trainG_clean (1).xlsx


In [ ]:
import io 
import pandas as pd
trainG = pd.read_excel(io.BytesIO(uploaded['trainG_clean.xlsx'])) 
test = pd.read_excel(io.BytesIO(uploaded['test_clean.xlsx'])) 

In [ ]:
trainG

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my apl frie...
1,2,0,i missed the new moon trail...
2,3,1,omg its already o
3,4,0,omgaga im sooo im gunna cry i'...
4,5,0,i think mi bf is cheating on me ...
...,...,...,...
75175,99995,0,cupati it took me waaay too long to get your ...
75176,99996,0,cupcake seems like a repeating problem hop...
75177,99998,0,cupcake ya i thought so
75178,99999,1,cupcake dollie yes yes i'm glad you had mor...


In [ ]:
test = test[test['ItemID'].map(len) < 10]
test

,ItemID,SentimentText
0,1,is so sad for my APL frien...
1,2,httI missed the New Moon tr...
2,3,omg its already 7:30 :O;;;;;;;;;...
3,4,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,i think mi bf is cheating on me!!! ...
...,...,...
299037,299993,@MartinDubow Welcome to my friends list and th...
299038,299994,@MartinDubow yeah and I envy u. ;;;;;;;;;;;;;;...
299042,299998,@MartinezMayra I got u at work when I go buy o...
299043,299999,@martinfaux He certainly gives good interview ...


In [ ]:
data = trainG.copy()
data.drop('ItemID', axis=1, inplace=True)
print(data.isnull().any())
data['Sentiment'] = data['Sentiment'].astype('category')
print(type(data['Sentiment'][0]))
data['label_id'] = data['Sentiment'].cat.codes
data['label_id'].head()

Sentiment        False
SentimentText    False
dtype: bool
<class 'numpy.int64'>


0    0
1    0
2    1
3    0
4    0
Name: label_id, dtype: int8

In [ ]:
import re

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

data['SentimentText']  = data['SentimentText'].astype(str)
test['SentimentText'] = test['SentimentText'].astype(str)
data['SentimentText'] = preprocess_reviews(data['SentimentText'])

test['SentimentText'] = preprocess_reviews(test['SentimentText'])

In [ ]:
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import classification_report,accuracy_score
import six.moves.cPickle as pickle


In [ ]:

import numpy as np 
import pandas as pd

import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy

In [ ]:
nlp=spacy.load("en_core_web_sm")

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
import string
punct=string.punctuation

def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = LinearSVC()

In [ ]:
x = trainG['SentimentText']
y = trainG['Sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [ ]:
clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_cleaning at 0x7fa144c6c6a8>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.67      0.69      6692
           1       0.75      0.77      0.76      8344

    accuracy                           0.73     15036
   macro avg       0.73      0.72      0.72     15036
weighted avg       0.73      0.73      0.73     15036



In [ ]:
y_pred=clf.predict(test['SentimentText'])

In [ ]:
y_pred

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
y_pred[1]

0

In [ ]:
sub_file=pd.DataFrame({'id':test['ItemID'],'target':y_pred.round().astype(int)})

In [ ]:
prediction = list() 
for i in range(len(test['SentimentText'])):
  prediction.append(y_pred[i]) 


test['prediction'] = prediction  

In [ ]:
from google.colab import files


test.to_excel('test_clean_pred.xlsx' , index=False)
files.download('test_clean_pred.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>